# BP神经网络收入预测

## 数据预处理

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
path = r'D:\JWE\Files\Courses\数据挖掘\work\第七-九章\data'
file_train = 'train.csv'
file_test = 'test.csv'
file_answer = 'correct_answer.csv'

train = pd.read_csv(path + '\\' + file_train)
test = pd.read_csv(path + '\\' + file_test)
answer = pd.read_csv(path + '\\' + file_answer)
income_data = pd.concat([train.copy(), test.copy()], sort=False)

In [4]:
data0 = pd.get_dummies(income_data)
train_onehot = data0.iloc[:32561,:]
test_onehot = data0.iloc[32561:,:108]

## BP using keras

In [3]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.utils import to_categorical
from keras.wrappers.scikit_learn import KerasClassifier

Using TensorFlow backend.


In [8]:
X_train = train_onehot.iloc[:,:108]
Y_train = train_onehot.iloc[:,108:]
X_test = test_onehot

In [9]:
def DNN_create(k=1,x_train=X_train,y_train=Y_train,units=50,divide=0.2,batch_size=30,epochs=200,drop_rate=0.5):
    long = len(x_train.head(1).values[0])
    DNN = Sequential()
    DNN.add(Dense(units=units,input_dim=long,activation='relu')) # 输入层
    # droupout层，减少overfiting
    for i in range(k):
#         if (k+1)%2:
#             DNN.add(Dropout(drop_rate))
        DNN.add(Dropout(drop_rate))
        DNN.add(Dense(units=units,activation='relu'))
    DNN.add(Dense(units=2,activation='softmax')) # 输出层
    DNN.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy']) # 'categorical_crossentropy' 多分类
    DNN.fit(x_train,y_train,batch_size=batch_size,epochs=epochs,validation_split=divide)
    return DNN

In [35]:
DNN = DNN_create(k=2,drop_rate=0.5,x_train=X_train,y_train=Y_train,units=100,batch_size=2000,epochs=100)

Train on 26048 samples, validate on 6513 samples
Epoch 1/100
26048/26048 [==============================] - 0s 18us/step - loss: 4.1770 - accuracy: 0.7282 - val_loss: 3.7677 - val_accuracy: 0.7543
Epoch 2/100
26048/26048 [==============================] - 0s 12us/step - loss: 3.6895 - accuracy: 0.7601 - val_loss: 3.7677 - val_accuracy: 0.7543
Epoch 3/100
26048/26048 [==============================] - 0s 13us/step - loss: 3.6865 - accuracy: 0.7603 - val_loss: 3.7677 - val_accuracy: 0.7543
Epoch 4/100
26048/26048 [==============================] - 0s 15us/step - loss: 3.6860 - accuracy: 0.7603 - val_loss: 3.7677 - val_accuracy: 0.7543
Epoch 5/100
26048/26048 [==============================] - 0s 14us/step - loss: 3.6865 - accuracy: 0.7603 - val_loss: 3.7677 - val_accuracy: 0.7543
Epoch 6/100
26048/26048 [==============================] - 0s 14us/step - loss: 3.6854 - accuracy: 0.7604 - val_loss: 3.7677 - val_accuracy: 0.7543
Epoch 7/100
26048/26048 [==============================] - 0s 1

Epoch 56/100
26048/26048 [==============================] - 0s 11us/step - loss: 3.6854 - accuracy: 0.7604 - val_loss: 3.7677 - val_accuracy: 0.7543
Epoch 57/100
26048/26048 [==============================] - 0s 10us/step - loss: 3.6860 - accuracy: 0.7603 - val_loss: 3.7677 - val_accuracy: 0.7543
Epoch 58/100
26048/26048 [==============================] - 0s 12us/step - loss: 3.6842 - accuracy: 0.7604 - val_loss: 3.7677 - val_accuracy: 0.7543
Epoch 59/100
26048/26048 [==============================] - 0s 11us/step - loss: 3.6860 - accuracy: 0.7603 - val_loss: 3.7677 - val_accuracy: 0.7543
Epoch 60/100
26048/26048 [==============================] - 0s 11us/step - loss: 3.6860 - accuracy: 0.7603 - val_loss: 3.7677 - val_accuracy: 0.7543
Epoch 61/100
26048/26048 [==============================] - 0s 11us/step - loss: 3.6854 - accuracy: 0.7604 - val_loss: 3.7677 - val_accuracy: 0.7543
Epoch 62/100
26048/26048 [==============================] - 0s 12us/step - loss: 3.6854 - accuracy: 0.7604

In [49]:
DNN.evaluate(X_train,Y_train)

result = DNN.predict(X_test.values)

32561/32561 [==============================] - 1s 21us/step


[3.693318721531306, 0.7591904401779175]

In [50]:
DNN_output = pd.DataFrame({'id':[i+1 for i in range(len(result))],'label':[1 if result[i,1] >= 1 else 0 for i in range(len(result))]})
DNN_output.to_csv(r'D:\JWE\Files\Courses\数据挖掘\work\第七-九章\DNN_output.csv',index=False)

数据预处理仍然很重要，没有合适的处理，结果会很坏。